# Snuffelfiets

In [ ]:
# Generieke imports, variabelen en functies.

from pathlib import Path
import numpy as np
import pandas as pd
from snuffelfiets import inlezen, opschonen, analyse, plotting

api_key = ''  # voeg hier de API key toe
output_map = ''  # voeg hier de data directory toe
sep_output = ';'
decimal_output = '.'

start_datum = '2023-01-01'
stop_datum = '2023-03-01'

def build_path(output_map, output_fn, start_datum, stop_datum):
    return Path(output_map, f'{output_fn}{start_datum}_tot_{stop_datum}.csv')


## Inlezen van de data uit de database

TODO ... beschrijving ...

In [ ]:
# Lees de data in uit de databasebase.
df = inlezen.call_api(api_key, start_datum, stop_datum)

# Schrijf de data naar een csv.
output_path_list = [output_map, 'api_gegevens_', start_datum, stop_datum]
output_file_path = build_path(*output_path_list)
df.to_csv(output_file_path, sep=sep_output, decimal=decimal_output, index=False)


## Opschonen van de data

In [ ]:
# Lees de data uit de csv.
output_path_list = [output_map, 'api_gegevens_', start_datum, stop_datum]
output_file_path = build_path(*output_path_list)
df = pd.read_csv(output_file_path, sep=sep_output, decimal=decimal_output)

# Corrigeer kolommen met default settings.
df = opschonen.correct_units(df)

# Verwijder ongeldige records
df = opschonen.verwijder_errors(df)  # , error_codes=[1, 4, 8, 16, 512, 1024, 4096]

# Schrijf de schone data naar een csv.
output_path_list = [output_map, 'clean_gegevens_', start_datum, stop_datum]
output_file_path = build_path(*output_path_list)
df.to_csv(output_file_path, sep=sep_output, decimal=decimal_output, index=False)


## Analyse van de data.

In [ ]:
# Lees de data uit de csv.
output_path_list = [output_map, 'clean_gegevens_', start_datum, stop_datum]
output_file_path = build_path(*output_path_list)
df = pd.read_csv(output_file_path, sep=sep_output, decimal=decimal_output)
df.shape


### Filteren van kolommen en rijen

In [ ]:
# Reduceer dataframe tot kolommen in de lijst.
cols = ['entity_id', 'recording_timestamp', 'latitude', 'longitude', 'pm2_5']
df = opschonen.filter_columns(df, cols)
list(df.columns)


### Bereken basisinfo uit ruwe data

In [ ]:
df = analyse.verdeel_in_ritten(df, t_seconden=30*60)
df = analyse.bereken_afstanden(df)
np.unique(df.rit_id)


## Plot de resultaten

### Maak een hexbin plot

In [ ]:
# de default plot
fig = plotting.hexbin_mapbox(df)
fig.show()


In [ ]:
# save in verschillende formats
output_path = Path(output_map, 'hexbin').expanduser()
formats = ['html', 'pdf', 'jpg']
plotting.save_fig(fig, output_path, formats)


In [ ]:
# voorbeeld van een override van de default settings
fig = plotting.hexbin_mapbox(
    df,
    hexbin_args={'color_continuous_scale': ['blue', 'red']},
    layout_args={'mapbox_style': 'open-street-map'},
    )
fig.show()
